In [2]:
import os
import oracledb
import requests

# Enable thick mode (uses Oracle Instant Client)
oracledb.init_oracle_client(lib_dir=r"C:\oracle\instantclient_21_15")

# Database connection details
hostname = "10.0.11.59"
port = 1521
service_name = "RMEDB"
username = "RME_DEV"
password = "PASS21RME"

# Microsoft Teams webhook URL
teams_webhook_url = "https://rowadmodern.webhook.office.com/webhookb2/84557fde-c0e1-457d-99d7-152442e5b0ac@7c9607e1-cd01-4c4f-a163-c7f2bb6284a4/IncomingWebhook/2ea28de8d34b417b8ded9774788916fd/24f28753-9c07-40e0-91b2-ea196c200a33/V2f9usIcwbZeFI--I8y4MnuI-SVjXJVlrVqccubKwdsIg1"

# File paths for storing last cash-in and last message
base_dir = r"D:\OneDrive\OneDrive - Rowad Modern Engineering\x004 Data Science\03.rme.db\00.repo\rme.db\13.airflow-spark\07.cash-in-message"
os.makedirs(base_dir, exist_ok=True)
last_cash_in_file = os.path.join(base_dir, "last_cash_in.txt")
last_message_file = os.path.join(base_dir, "last_message.txt")

# Helper functions to read/write from/to text files
def read_file(file_path):
    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as file:
            return file.read().strip()
    return ""

def write_file(file_path, content):
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(content)

try:
    print("Connecting to the database...")
    # Connect to the database
    dsn = oracledb.makedsn(hostname, port, service_name=service_name)
    connection = oracledb.connect(user=username, password=password, dsn=dsn)
    print("Connected to the database!")

    # Create a cursor
    cursor = connection.cursor()

    # Query to get the latest cash-in transaction
    query = """
    SELECT TO_CHAR(acr.RECEIPT_DATE, 'YYYY-MM-DD') AS receipt_date,
           TRUNC(acr.amount * NVL(acr.exchange_rate, 1)) AS Recipt_amount_EGP,
           acr.comments
    FROM AR_CASH_RECEIPTS_ALL acr
    WHERE acr.ATTRIBUTE_CATEGORY = '83'
      AND acr.ATTRIBUTE1 IS NOT NULL
      AND acr.ATTRIBUTE2 != 'Manual/Netting'
    ORDER BY acr.RECEIPT_DATE DESC
    FETCH FIRST 1 ROW ONLY
    """
    cursor.execute(query)
    result = cursor.fetchone()

    if result:
        # Extract the current transaction details
        receipt_date, amount_egp, comment = result
        current_transaction = f"{receipt_date}|{amount_egp}|{comment.strip()}"

        print(f"Fetched transaction from database: {current_transaction}")

        # Read the last cash-in and last message from files
        last_transaction = read_file(last_cash_in_file)
        last_message = read_file(last_message_file)
        print(last_transaction)
        print(current_transaction)
        # Check if the transaction is new
        if current_transaction != last_transaction:
            print("New transaction found! Preparing to send to Teams...")
            # Construct the message
            message = (
                f"New Cash-In Transaction:\n"
                f"📅 Date: {receipt_date}\n"
                f"💰 Amount: {amount_egp:,} EGP\n"
                f"📝 Comment: {comment}"
            )

            # Check if the message is already sent
            if message != last_message:
                # Send the message to Microsoft Teams
                payload = {"text": message}
                response = requests.post(teams_webhook_url, json=payload, verify=False)

                if response.status_code == 200:
                    print("Message sent to Microsoft Teams successfully!")

                    # Update the files
                    write_file(last_cash_in_file, current_transaction)
                    write_file(last_message_file, message)
                else:
                    print(f"Failed to send message. Status code: {response.status_code}, Response: {response.text}")
            else:
                print("Message already sent. No duplicate message sent to Teams.")
        else:
            print("No new cash-in transactions found.")
    else:
        print("No cash-in transactions found in the database.")

except oracledb.Error as error:
    print(f"Error connecting to database: {error}")

finally:
    # Close the cursor and connection
    try:
        if 'cursor' in locals() and cursor is not None:
            cursor.close()
    except oracledb.Error as error:
        print(f"Error while closing cursor: {error}")

    try:
        if 'connection' in locals() and connection is not None:
            connection.close()
    except oracledb.Error as error:
        print(f"Error while closing connection: {error}")


Connecting to the database...
Connected to the database!
Fetched transaction from database: 2025-02-06|3917447|ipc 39 - ORA ZED BARK ph2.
2025-02-06|3917447|ipc 39 - ORA ZED BARK ph2.
2025-02-06|3917447|ipc 39 - ORA ZED BARK ph2.
No new cash-in transactions found.
